In [1]:
from sklearn.datasets import load_wine

In [2]:
wine = load_wine()

In [ ]:
print(wine.DESCR)

In [3]:
x = wine.data
y = wine.target

In [4]:
x.shape, y.shape

((178, 13), (178,))

* 조건 : Functional API
    1. training set / test set / validation set 구분하기!
    2. scaling (min-max, standardization 둘 중!)
    3. y에 대한 전처리
    4. 히든 레이어 최소 2개 쓰기 (노드와 액티베이션 자유!)
    5. early stopping, validation_data

In [6]:
## 전처리 : X에 대하여
# split
from sklearn.model_selection import train_test_split

In [7]:
train_x, test_x, train_y, test_y =\
    train_test_split(x, y, test_size=0.1, random_state=2023)

train_x, val_x, train_y, val_y =\
    train_test_split(train_x, train_y, test_size=0.1, random_state=2023)

In [8]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((144, 13), (144,), (16, 13), (16,))

In [9]:
## scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
mm_scaler, sd_scaler = MinMaxScaler(), StandardScaler()

In [24]:
import numpy as np

In [ ]:
pd.DataFrame((train_x - np.min(train_x, axis=0))/(np.max(train_x, axis=0)-np.min(train_x, axis=0))).describe()

In [ ]:
pd.DataFrame((train_x - np.mean(train_x, axis=0))/(np.std(train_x, axis=0))).describe()

In [12]:
import pandas as pd

In [ ]:
pd.DataFrame(train_x, columns=wine.feature_names).describe()

In [14]:
train_x_mm = mm_scaler.fit_transform(train_x)
train_x_sd = sd_scaler.fit_transform(train_x)

In [ ]:
pd.DataFrame(train_x_mm, columns=wine.feature_names).describe()

In [ ]:
pd.DataFrame(train_x_sd, columns=wine.feature_names).describe()

In [17]:
val_x_sd = sd_scaler.transform(val_x)
test_x_sd = sd_scaler.transform(test_x)

In [18]:
## Y에 대한 전처리
from tensorflow.keras.utils import to_categorical

In [22]:
class_n = len(set(y))

In [23]:
train_y = to_categorical(train_y, class_n)
val_y = to_categorical(val_y, class_n)
test_y = to_categorical(test_y, class_n)

In [42]:
train_x_sd.shape, train_y.shape

((144, 13), (144, 3))

In [37]:
import tensorflow as tf
from tensorflow import keras

In [39]:
## 모델링 : Functional API
# 1번 세션 클리어
keras.backend.clear_session()

# 2번 레이어 엮기
il = keras.layers.Input(shape=(13,))
hl = keras.layers.Dense(64, activation='relu', name='hidden1')(il)
hl = keras.layers.Dense(32, activation='relu', name='hidden2')(hl)
ol = keras.layers.Dense(3, activation='softmax', name='output')(hl)

# 3번 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

# 4번 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 hidden1 (Dense)             (None, 64)                896       
                                                                 
 hidden2 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 3)                 99        
                                                                 
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [40]:
## early stopping
from tensorflow.keras.callbacks import EarlyStopping

In [41]:
es = EarlyStopping(monitor='val_loss',         # 관측 대상
                   min_delta=0,                # 임계값
                   patience=5,                 # 관측 대상의 성능 개선이 발생하지 않을 때, 얼마나 더 지켜볼 것인지
                   verbose=1,
                   restore_best_weights=True)  # 최적의 epoch 가중치로 적용

In [43]:
# 학습
model.fit(train_x_sd, train_y, epochs=10000, verbose=1,
          callbacks=[es], validation_data=(val_x_sd, val_y))

Epoch 1/10000
5/5 [==============================] - 2s 90ms/step - loss: 1.0071 - accuracy: 0.4236 - val_loss: 0.8944 - val_accuracy: 0.5000
Epoch 2/10000
5/5 [==============================] - 0s 17ms/step - loss: 0.8722 - accuracy: 0.6181 - val_loss: 0.7828 - val_accuracy: 0.7500
Epoch 3/10000
5/5 [==============================] - 0s 16ms/step - loss: 0.7584 - accuracy: 0.8056 - val_loss: 0.6811 - val_accuracy: 0.8125
Epoch 4/10000
5/5 [==============================] - 0s 17ms/step - loss: 0.6601 - accuracy: 0.8819 - val_loss: 0.5826 - val_accuracy: 0.8125
Epoch 5/10000
5/5 [==============================] - 0s 13ms/step - loss: 0.5697 - accuracy: 0.8958 - val_loss: 0.4903 - val_accuracy: 0.9375
Epoch 6/10000
5/5 [==============================] - 0s 17ms/step - loss: 0.4874 - accuracy: 0.9306 - val_loss: 0.4071 - val_accuracy: 0.9375
Epoch 7/10000
5/5 [==============================] - 0s 17ms/step - loss: 0.4122 - accuracy: 0.9375 - val_loss: 0.3341 - val_accuracy: 1.0000
Epoch 

In [45]:
y_pred = model.predict(test_x_sd)

1/1 [==============================] - 0s 330ms/step


In [46]:
y_pred[:5]

array([[2.8838762e-04, 1.1958459e-02, 9.8775309e-01],
       [3.6925497e-04, 9.9946606e-01, 1.6479444e-04],
       [1.6824119e-03, 7.1565025e-03, 9.9116117e-01],
       [8.0794276e-04, 9.9850571e-01, 6.8637641e-04],
       [9.9456328e-01, 4.0324223e-03, 1.4042574e-03]], dtype=float32)

In [48]:
test_y[:5]

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)